In [60]:
import numpy as np
import random

In [61]:
def dataGenerator(n):
    Data = np.zeros((n, 8), dtype=int)
    StrData = "11"  # Start delimiter
    for i in range(n):
        rand_num = random.randint(0, 255)  
        lstr = bin(rand_num)[2:].zfill(8)  # 8-bit binary string
        oddParity = (lstr.count("1")) % 2  # Parity bit (unused here)
        StrData += "0" + lstr + "1" + "11"  # Frame: start bit + data + end bit + delimiter
        lst = list(map(int, lstr))  
        Data[i, 0:8] = lst  
    return StrData

In [62]:
'''
FIFO (Circular Replacement)
    ~ allows Duplicates
    ~ Owerwrites the oldest data in FIFO with the newest data

'''
Str = dataGenerator(11)

FIFO = np.zeros((8, 8), dtype=int)

j = 2  # Start after initial '11'
m = 0  # FIFO column index

while j + 12 <= len(Str):
    # Assuming each frame is of size: 1 (start) + 8 (data) + 1 (end) + 2 (delimiter) = 12 bits
    
    start_bit = Str[j]          # Should be '0'
    data_bits = Str[j + 1: j + 9]  # 8 bits of data
    end_bit = Str[j + 9]        # Should be '1'
    delimiter = Str[j + 10: j + 12]  # Should be '11'
    
    # Convert data_bits (string) to list of ints
    FIFO[m, :] = list(map(int, data_bits))
    
    # Move to next frame
    j += 12
    m += 1
    if m == 8:
        m = 0

print("StrData:", Str)
print("FIFO Buffer:\n", FIFO)

StrData: 11001010011111010101010111000110011111010110001111001101001111011111010111010010011111001100000111011110001111011110000111000100000111
FIFO Buffer:
 [[1 1 1 1 0 0 0 1]
 [1 1 1 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [1 0 1 1 0 0 0 1]
 [0 1 1 0 1 0 0 1]
 [1 1 1 1 1 0 1 0]
 [1 0 0 1 0 0 1 1]
 [0 1 1 0 0 0 0 0]]


In [65]:
'''
FIFO (Least Recently Used Replacement)
    ~ Doesn't Allow Duplicates
    ~ If it gets a duplicate input, it pushes the input to the Front (Most Recently Used)
    ~ Overwrites the Oldest Data or The Least Recently Used (LRU) data in FIFO with the newest data
    ~ Newest Data is pushed to the Front
'''

# Data Frames (Each of 8 bits)
data_strings = [
    "10101010",  # Frame 1
    "11110000",  # Frame 2
    "00001111",  # Frame 3
    "11001100",  # Frame 4
    "00110011",  # Frame 5
    "01010101",  # Frame 6
    "10011001",  # Frame 7
    "11111111",  # Frame 8
    "10101010",  # Frame 9 (duplicate of Frame 1)
    "00000000",  # Frame 10
    "11110000",  # Frame 11 (duplicate of Frame 2)
    "01010101",  # Frame 12 (duplicate of Frame 6)
]

# Generate properly structured StrData
# Frame format: Start Bit (0) + 8 Data Bits + End Bit (1) + Delimiter (11)
StrData = "11"  # Initial delimiter to mark the beginning
for frame in data_strings:
    StrData += "0" + frame + "1" + "11"

print("StrData:", StrData)

# FIFO Buffer (Initially empty), using list for LRU behavior
FIFO = []

# Start parsing frames from index 2 (skipping initial delimiter)
j = 2

# Process all frames in the StrData
while j + 12 <= len(StrData):
    # Extract components of the frame
    start_bit = StrData[j]                     # Should be '0'
    data_bits = StrData[j + 1: j + 9]         # 8 data bits
    end_bit = StrData[j + 9]                  # Should be '1'
    delimiter = StrData[j + 10: j + 12]       # Should be '11'

    # Sanity check (optional, but good for debugging)
    if start_bit != '0' or end_bit != '1' or delimiter != '11':
        print(f"Invalid frame format at index {j}: Skipping this frame.")
        j += 12
        continue

    # Convert data_bits (string) to a list of integers
    data_list = list(map(int, data_bits))

    # LRU Logic
    if data_list in FIFO:
        # Duplicate: Move it to the front (Most Recently Used)
        FIFO.remove(data_list)
        FIFO.insert(0, data_list)
    else:
        # New entry: Add to front (Most Recently Used)
        if len(FIFO) == 8:
            # Remove Least Recently Used (last item)
            FIFO.pop()
        FIFO.insert(0, data_list)

    # Move to next frame
    j += 12

# Convert FIFO to a NumPy array for nicer display
FIFO_array = np.array(FIFO)

# Final Output
print("Final FIFO Buffer (Most Recently Used at Top):")
print(FIFO_array)


StrData: 11010101010111011110000111000001111111011001100111000110011111001010101111010011001111011111111111010101010111000000000111011110000111001010101111
Final FIFO Buffer (Most Recently Used at Top):
[[0 1 0 1 0 1 0 1]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1]
 [1 0 0 1 1 0 0 1]
 [0 0 1 1 0 0 1 1]
 [1 1 0 0 1 1 0 0]]


In [64]:
import numpy as np

'''
FIFO (Second Chance Replacement)
    ~ Allows duplicates, but gives a second chance to recently used frames.
    ~ Uses a reference bit to decide whether to replace a frame.
    ~ Overwrites the oldest frame (FIFO), but gives a second chance if its ref bit is 1.
    ~ If data is already in the buffer, it sets its ref bit to 1.
'''

# Data Frames (Each of 8 bits)
data_strings = [
    "10101010",  # Frame 1
    "11110000",  # Frame 2
    "00001111",  # Frame 3
    "11001100",  # Frame 4
    "00110011",  # Frame 5
    "01010101",  # Frame 6
    "10011001",  # Frame 7
    "11111111",  # Frame 8
    "10101010",  # Frame 9 (duplicate of Frame 1)
    "00000000",  # Frame 10
    "11110000",  # Frame 11 (duplicate of Frame 2)
    "01010101",  # Frame 12 (duplicate of Frame 6)
]

# Generate properly structured StrData
# Frame format: Start Bit (0) + 8 Data Bits + End Bit (1) + Delimiter (11)
StrData = "11"  # Initial delimiter to mark the beginning
for frame in data_strings:
    StrData += "0" + frame + "1" + "11"

print("StrData:", StrData)

# FIFO Buffer (Initially empty) and reference bits
FIFO = []        # To store frames (lists of 8 bits)
ref_bits = []    # To store reference bits for each frame
pointer = 0      # Points to the next candidate for replacement (clock hand)

# Start parsing frames from index 2 (skipping initial delimiter)
j = 2

# Process all frames in the StrData
while j + 12 <= len(StrData):
    # Extract components of the frame
    start_bit = StrData[j]                     # Should be '0'
    data_bits = StrData[j + 1: j + 9]         # 8 data bits
    end_bit = StrData[j + 9]                  # Should be '1'
    delimiter = StrData[j + 10: j + 12]       # Should be '11'

    # Sanity check (optional)
    if start_bit != '0' or end_bit != '1' or delimiter != '11':
        print(f"Invalid frame format at index {j}: Skipping this frame.")
        j += 12
        continue

    # Convert data_bits (string) to a list of integers
    data_list = list(map(int, data_bits))

    # Second Chance Algorithm Logic
    if data_list in FIFO:
        # Data already exists: Give second chance (set ref bit to 1)
        index = FIFO.index(data_list)
        ref_bits[index] = 1
        print(f"Frame {data_list} already exists. Giving second chance (ref bit set to 1).")
    else:
        if len(FIFO) < 8:
            # Space available: Add the new frame
            FIFO.append(data_list)
            ref_bits.append(0)
            print(f"Added new frame {data_list} at position {len(FIFO)-1}.")
        else:
            # Buffer is full: Find frame to replace using second chance
            replaced = False
            while not replaced:
                # If reference bit is 0, replace this frame
                if ref_bits[pointer] == 0:
                    print(f"Replacing frame at position {pointer} with {data_list}.")
                    FIFO[pointer] = data_list
                    ref_bits[pointer] = 1
                    replaced = True
                else:
                    # Give second chance: reset reference bit and move pointer
                    print(f"Frame at position {pointer} given second chance (ref bit reset).")
                    ref_bits[pointer] = 0
                # Move pointer (circular)
                pointer = (pointer + 1) % 8

    # Move to next frame
    j += 12

# Convert FIFO to a NumPy array for nicer display
FIFO_array = np.array(FIFO)

# Final Output
print("\nFinal FIFO Buffer (Order as per Second Chance Algorithm):")
print(FIFO_array)


StrData: 11010101010111011110000111000001111111011001100111000110011111001010101111010011001111011111111111010101010111000000000111011110000111001010101111
Added new frame [1, 0, 1, 0, 1, 0, 1, 0] at position 0.
Added new frame [1, 1, 1, 1, 0, 0, 0, 0] at position 1.
Added new frame [0, 0, 0, 0, 1, 1, 1, 1] at position 2.
Added new frame [1, 1, 0, 0, 1, 1, 0, 0] at position 3.
Added new frame [0, 0, 1, 1, 0, 0, 1, 1] at position 4.
Added new frame [0, 1, 0, 1, 0, 1, 0, 1] at position 5.
Added new frame [1, 0, 0, 1, 1, 0, 0, 1] at position 6.
Added new frame [1, 1, 1, 1, 1, 1, 1, 1] at position 7.
Frame [1, 0, 1, 0, 1, 0, 1, 0] already exists. Giving second chance (ref bit set to 1).
Frame at position 0 given second chance (ref bit reset).
Replacing frame at position 1 with [0, 0, 0, 0, 0, 0, 0, 0].
Replacing frame at position 2 with [1, 1, 1, 1, 0, 0, 0, 0].
Frame [0, 1, 0, 1, 0, 1, 0, 1] already exists. Giving second chance (ref bit set to 1).

Final FIFO Buffer (Order as per Second C